# init

In [ ]:
from quantifiers.CC import classify_count
from quantifiers.ACC import ACC
from quantifiers.dys_method import dys_method
from quantifiers.MS import MS_method

from utils.getTrainingScores import getTrainingScores
from utils.getTPRFPR import getTPRFPR
from utils.applyquantifiers import apply_quantifier
from utils.fitQuantifierSchumacherGithub import fitQuantifierSchumacherGithub

import pdb
import quapy as qp
import os
import numpy as np
import pandas as pd
import joblib
from scipy.io.arff import loadarff
from pprint import pprint
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

meta_table_path = './metafeatures/meta-table.csv'
path = "./datasets/"
files = os.listdir(path)


# counters = ["CC","ACC","SMM","HDy","DyS","SORD","MS","MS2","MAX","X","T50"]
counters = ["CC","ACC","SMM","HDy","DyS","SORD","MS","MS2","MAX","X","T50","PCC","PACC","GAC","GPAC","FM"]

# preprocess

In [ ]:
i = 0
dataframe = None
X = None
y = None
X_list = []
y_list = []

for f in files:
  df = pd.read_csv(path + f)
  df = df.dropna()

  y = df.pop(df.columns[-1])
  X = df

  y_list.append(y.to_numpy())
  X_list.append(X.to_numpy())

  i += 1
i = 0

In [ ]:
meta_table = pd.read_csv(meta_table_path)
meta_table_dict = {}

for counter in counters:
    meta_table_dict[counter] = meta_table.copy(deep = True)

meta_table

# meta-features

In [ ]:
def run_experiment(X, y, dataset_name):
  #......................input/output path directories....................

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
  rf_clf = None
  rf_clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
  # try:
  #   rf_clf = joblib.load('./models/' + dataset_name + '.joblib')
  #   rf_clf.n_jobs = -1
  # except:
  #   rf_clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

  scores = getTrainingScores(X_train, y_train, 10, rf_clf)[0] # None
  tprfpr = getTPRFPR(scores)
  rf_clf.fit(X_train, y_train) 

  niterations = 10 # how many replicates it will take
  batch_sizes = list([50]) # list(range(10, min(91, max_allowed + 1), 10))# + list(range(100, min(501, max_allowed + 1), 100)) # test set sizes
  alpha_values = [round(x, 2) for x in np.linspace(0,1,20)]   # class proportion

  pos_scores = scores[scores["class"]==1]["scores"]
  neg_scores = scores[scores["class"]==0]["scores"]
  
  X_test = pd.DataFrame(X_test)
  y_test = pd.DataFrame(y_test, columns=[str(len(X_test.columns))])
  df_test = pd.concat([X_test, y_test], axis=1)
  
  # WAS ZERO (0) BEFORE
  df_test_pos = df_test.loc[df_test[df_test.columns[-1]] == 1] # seperating positive test examples
  df_test_neg = df_test.loc[df_test[df_test.columns[-1]] == 0] # seperating negative test examples
  
  # pdb.set_trace()

  table=pd.DataFrame(columns=['quantifier', 'abs-error'])
  for sample_size in batch_sizes:   # [10,100,500], batch_sizes, Varying test set sizes

    for alpha in alpha_values: # Varying positive class distribution
      # abs_error_cc = []
      # abs_error_dys = []

      error = []

      for iter in range(niterations):
        pos_size = int(round(sample_size * alpha, 2))
        neg_size = sample_size - pos_size

        #
        # AVISAR O PROF ANDRE QUE SÓ FUNCIONOU COM REPLACE = TRUE
        #
        # df_test_neg
        # sample_test_pos = df_test_pos.sample( int(pos_size), replace = False)
        # sample_test_neg = df_test_neg.sample( int(neg_size), replace = False)
        sample_test_pos = df_test_pos.sample( int(pos_size), replace = True)
        sample_test_neg = df_test_neg.sample( int(neg_size), replace = True)

        sample_test = pd.concat([sample_test_pos, sample_test_neg])

        test_label = sample_test[sample_test.columns[-1]] # sample_test["class"]

        test_sample = sample_test.drop([sample_test.columns[-1]], axis=1) # sample_test.drop(["class"], axis=1)  #dropping class label columns
        te_scores = rf_clf.predict_proba(test_sample)[:,1]  #estimating test sample scores

        n_pos_sample_test = list(test_label).count(1) #Counting num of actual positives in test sample
        calcultd_pos_prop = round(n_pos_sample_test/len(sample_test), 2) #actual pos class prevalence in generated sample

        # print(counters)
        for quantifier in counters:
          #..............Test Sample QUAPY exp...........................
          te_quapy = None
          external_qnt = None
          #if quantifier in ['EM', 'PWK']:
          #  print('ok')
          #  external_qnt = fitQuantifierSchumacherGithub(quantifier, X_train, y_train)                  
          #  te_quapy = qp.data.LabelledCollection(sample_test.drop(["class","Binary_label"], axis=1), test_label)
          
          #.............Calling of Methods..................................................
          pred_pos_prop = apply_quantifier(qntMethod=quantifier,
                                           clf=None,
                                           scores=scores,
                                           p_score=pos_scores,
                                           n_score=neg_scores,
                                           train_labels=None,
                                           test_score=te_scores,
                                           TprFpr=tprfpr,
                                           thr=0.5,
                                           measure='hellinger',
                                           test_data=test_sample,
                                           test_quapy=te_quapy,
                                           external_qnt=external_qnt) #y_test=test_label
          
          pred_pos_prop = np.round(pred_pos_prop,2)  #predicted class proportion
          
          #..............................RESULTS Evaluation.....................................
          abs_error = round(abs(calcultd_pos_prop - pred_pos_prop), 2) # absolute error

          table.loc[len(table)] = [quantifier,abs_error]
          # error = round(calcultd_pos_prop - pred_pos_prop, 2)     # simple error Biasness
          
  return table

In [ ]:
clf = RandomForestClassifier(random_state=42)
skip_count = 0
table = None
# warnings.filterwarnings('ignore')

result = {}
for counter in counters:
    result[counter] = []

file = open('log.txt', 'w')
file.close()

for i in range(0, len(X_list)):
  try:
    clf = RandomForestClassifier()

    table = run_experiment(X_list[i], y_list[i], str(files[i].split('.csv')[0]))
    table = table.groupby('quantifier')['abs-error'].aggregate('mean')

    for key in counters:
      result[key].append(table[key])
 
    print('Finished ' + str(i))
  except Exception as e:
    print('Skipping ' + str(i) + '...\t\t\t' + str(e))
    for key in meta_table_dict:
      meta_table_dict[key].drop(i, inplace = True)
    skip_count += 1

    file = open('log.txt', 'a')
    file.write('Skipping ' + str(i) + '...\t\t\t' + str(e) + '\n')
    file.close()

  # # TEST
  # if i == 20:
  #   for key in meta_table_dict:
  #     meta_table_dict[key].reset_index(drop=True, inplace=True)
  #     meta_table_dict[key].drop(meta_table_dict[key].index[((i+1)-skip_count):], inplace=True)
  #   break

for key in meta_table_dict:
  meta_table_dict[key]['abs-error'] = result[key]

print('\n\nSkipped ' + str(skip_count) + ' dataset(s)')

In [ ]:
# result

In [ ]:
# meta_table_dict

In [ ]:
finSavePath = './metafeatures/meta-table'
for c in counters:
    meta_table_dict[c].to_csv(str(finSavePath + '-' + c + '.csv'), index = False)
# test = pd.read_csv('./metafeatures/meta-table-DyS.csv')
# test

In [ ]:
# test = pd.read_csv('./metafeatures/meta-table-MS2.csv')
# test = test.pop(test.columns[-1])
# test

In [ ]:
# i = 0
# for t in test:
#     if np.isnan(t):
#         print(i)
#     i += 1